In [1]:
import omnipy as om
from omnipy import Dataset, Model
om.setup_jupyter_ui()

ℹ️ 
 Omnipy has detected that it is being imported into a Jupyter
session. Please run the following in a separate cell:

 import omnipy as om
om.setup_jupyter_ui()

🔌 
 The Jupyter kernel currently runs in embedded mode, e.g. from within PyCharm. (If you are running Jupyter in a web browser, you probably want to restart the kernel by clicking the ↻ icon in the Jupyter toolbar.)

⚠️ 
 This cell was not loaded properly! You probably want to rerun the code cell above ( Click in the code cell, and press Shift+Enter ⇧ + ↩ ).

No Omnipy-related CSS styling was loaded due to plain text output.

↻ 
 Make sure to rerun this cell whenever the kernel is restarted.

In [47]:
om.runtime.config.data.ui.jupyter.color.style = om.LightHighContrastTintedThemingBase16ColorStyles.RANDOM_LIGHT_HIGH_CONTRAST_T16
om.runtime.config.data.ui.jupyter.color.dark_background = False
om.runtime.config.data.ui.jupyter.color.solid_background = True
om.runtime.config.root_log.log_to_stdout = False
om.runtime.config

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                 ConfigModel                                                  │
│                                                                                                              │
│ {                                                                                                            │
│   "data": {                                                                                                  │
│     "ui": {                                                                                                  │
│       "detected_type": "pycharm_jupyter",                                                                    │
│       "terminal": {                                                                                          │
│         "dims_mode": "auto",                                                                  

In [31]:
from nobel_api_output_model import NobelPrizesResult, LaureateBasic, NobelPrize

# Omnipy tasks
@om.TaskTemplate()
def create_nobel_prize_urls(category: str | None, page_size: int, num_pages: int) -> om.HttpUrlDataset:
    def _nobel_price_api_urls(category: str | None, offset: int, limit: int, ) -> om.HttpUrlModel:
        url = om.HttpUrlModel('https://api.nobelprize.org')
        url.path // '2.1' // 'nobelPrizes'
        if category:
            url.query['nobelPrizeCategory'] = category
        url.query['offset'] = offset
        url.query['limit'] = limit
        return url

    return om.HttpUrlDataset({
        page_offset: _nobel_price_api_urls(category=category, offset=page_offset, limit=page_size)
        for page_offset in range(0, num_pages * page_size, page_size)
    })

@om.TaskTemplate()
async def fetch_and_extract_laureates_per_year(nobel_prize_urls: om.HttpUrlDataset) -> Dataset[Model[list[LaureateBasic]]]:
    per_page_nobel_prize_info = await Dataset[Model[NobelPrizesResult]].load(nobel_prize_urls)
    
    per_year_nobel_prizes = Dataset[Model[list[LaureateBasic]]]()

    all_nobel_prizes = []
    for page in per_page_nobel_prize_info.values():
        all_nobel_prizes += page.nobelPrizes
        
    for entry in all_nobel_prizes:
        if entry.laureates:
            per_year_nobel_prizes[str(entry.awardYear)] = entry.laureates
    
    return per_year_nobel_prizes


# Omnipy dataflows
@om.LinearFlowTemplate(
    create_nobel_prize_urls,
    fetch_and_extract_laureates_per_year,
)
def download_and_wrangle_nobel_prizes(category: str | None, page_size: int, num_pages: int):
    ...

# Run the dataflow and show results
literature_prizes = await download_and_wrangle_nobel_prizes.run(category='lit', page_size=25, num_pages=6)
literature_prizes

╭────┬────────────────┬────────────────────────────┬────────┬──────────────────╮
│ #  │ Data file name │            Type            │ Length │ Size (in memory) │
│    │                │                            │        │                  │
│ 0  │ 1901           │ Model[list[LaureateBasic]] │      1 │           5.4 kB │
│ 1  │ 1902           │ Model[list[LaureateBasic]] │      1 │           5.3 kB │
│ 2  │ 1903           │ Model[list[LaureateBasic]] │      1 │           5.4 kB │
│ 3  │ 1904           │ Model[list[LaureateBasic]] │      2 │           9.4 kB │
│ 4  │ 1905           │ Model[list[LaureateBasic]] │      1 │           5.1 kB │
│ 5  │ 1906           │ Model[list[LaureateBasic]] │      1 │           5.4 kB │
│ 6  │ 1907           │ Model[list[LaureateBasic]] │      1 │           5.4 kB │
│ 7  │ 1908           │ Model[list[LaureateBasic]] │      1 │           5.5 kB │
│ 8  │ 1909           │ Model[list[LaureateBasic]] │      1 │           5.3 kB │
│ 9  │ 1910           │ Mode

In [22]:
literature_prizes[-3:].peek(style='rose-pine-dawn-t16', bg=True, syntax='json', h_overflow='wrap')

ValidationError: 1 validation error for ResizedLayoutDraftPanel
content
  Object of type Portion is not JSON serializable (type=type_error)

In [ ]:
(await om.JsonDataset.load(["https://api.nobelprize.org/2/laureate/1032"])).full()